In [4]:
import pickle
import numpy as np
import tensorflow as tf
# tf.python.control_flow_ops = tf

with open('small_traffic_set/small_train_traffic.p', mode='rb') as f:
    data = pickle.load(f)

X_train, y_train = data['features'], data['labels']

# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

# Build the Fully Connected Neural Network in Keras Here
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(32, 32, 3)))
#Hint 1: The Keras example of a convolutional neural network for MNIST would be a good example to review.
#Hint 2: You can set the padding type by passing in a border_mode= argument to the Convolution2D() layer.

model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))

# model = Sequential()
# model.add(Flatten(input_shape=(32, 32, 3)))
# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dense(5))
# model.add(Activation('softmax'))

# preprocess data
X_normalized = np.array(X_train / 255.0 - 0.5 )

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer() # 功能与OneHotEncoder相似，但可以处理非数值型数据
y_one_hot = label_binarizer.fit_transform(y_train)

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, epochs=3, validation_split=0.2)

C:\Tools\Anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3...)`


Train on 80 samples, validate on 20 samples
Epoch 1/3
80/80 [==============================] - 0s 5ms/step - loss: 1.3883 - acc: 0.3125 - val_loss: 0.8199 - val_acc: 0.4500
Epoch 2/3
80/80 [==============================] - 0s 2ms/step - loss: 0.8356 - acc: 0.5500 - val_loss: 0.6053 - val_acc: 0.8000
Epoch 3/3
80/80 [==============================] - 0s 1ms/step - loss: 0.6172 - acc: 0.7750 - val_loss: 0.4180 - val_acc: 0.8500


In [3]:
with open('small_traffic_set/small_test_traffic.p', 'rb') as f:
    data_test = pickle.load(f)

X_test = data_test['features']
y_test = data_test['labels']

# preprocess data
X_normalized_test = np.array(X_test / 255.0 - 0.5 )
y_one_hot_test = label_binarizer.fit_transform(y_test)

print("Testing")

metrics = model.evaluate(X_normalized_test, y_one_hot_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

Testing
20/20 [==============================] - 0s 500us/step
loss: 0.5408651232719421
acc: 0.75


In [ ]:
import tensorflow as tf
input = tf.placeholder(tf.float32, (None, 32, 32, 3))
filter_weights = tf.Variable(tf.truncated_normal((8, 8, 3, 20))) # (height, width, input_depth, output_depth)
filter_bias = tf.Variable(tf.zeros(20))
strides = [1, 2, 2, 1] # (batch, height, width, depth)
padding = 'SAME'
conv = tf.nn.conv2d(input, filter_weights, strides, padding) + filter_bias

##### In summary TensorFlow uses the following equation for 'SAME' vs 'VALID'

**SAME** Padding, the output height and width are computed as:

* out_height = ceil(float(in_height) / float(strides[1]))

* out_width = ceil(float(in_width) / float(strides[2]))

**VALID** Padding, the output height and width are computed as:

* out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))

* out_width = ceil(float(in_width - filter_width + 1) / float(strides[2]))

In [ ]:
# Output depth
k_output = 64

# Image Properties
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_height = 5

# Input/Image
input = tf.placeholder(
    tf.float32,
    shape=[None, image_height, image_width, color_channels])

# Weight and bias
weight = tf.Variable(tf.truncated_normal(
    [filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))

# Apply Convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

# Apply Max Pooling
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

The code above uses the **tf.nn.conv2d()** function to compute the convolution with weight as the filter and **[1, 2, 2, 1]** for the strides. TensorFlow uses a stride for each input dimension, **[batch, input_height, input_width, input_channels]**.

You'll focus on changing input_height and input_width while setting batch and input_channels to 1. 
This example code uses a stride of 2 with 5x5 filter over input.

The **tf.nn.bias_add()** function adds a 1-d bias to the last dimension in a matrix.

The **tf.nn.max_pool()** function performs max pooling with the ksize parameter as the size of the filter and the strides parameter as the length of the stride. **2x2** filters with a stride of **2x2** are common in practice.

The ksize and strides parameters are structured as **4-element** lists, with each element corresponding to a dimension of the input tensor **([batch, height, width, channels])**. For both ksize and strides, the batch and channel dimensions are typically set to **1**.

In [7]:
y_train

array([2, 2, 3, 2, 4, 3, 3, 3, 1, 4, 2, 4, 2, 5, 1, 4, 5, 1, 1, 1, 4, 2,
       1, 4, 5, 3, 4, 1, 3, 4, 4, 5, 5, 1, 4, 5, 2, 5, 1, 5, 3, 5, 2, 2,
       1, 3, 5, 5, 4, 2, 5, 4, 3, 3, 2, 5, 3, 2, 3, 3, 1, 5, 5, 2, 1, 2,
       2, 2, 4, 1, 5, 4, 4, 5, 5, 3, 1, 3, 4, 3, 4, 4, 5, 2, 4, 1, 3, 5,
       4, 4, 2, 5, 1, 1, 2, 3, 1, 3, 1, 4], dtype=uint8)